Import file & clean data

In [1]:
%matplotlib inline
import numpy as np
import plotly.express as px
import pandas as pd
import matplotlib.pyplot as plt
import requests
import descartes
import calendar
import seaborn as sns
import matplotlib
# Loading data (https://www.nti.org/analysis/articles/cns-north-korea-missile-test-database/). Encoding problem solved with 'unicode_escape'.
df = pd.read_csv('../data/north_korea_missile_test_database.csv', encoding= 'unicode_escape', index_col= "F1")
# Dropping unused columns.
df = df.drop(columns=['Launch Agency/Authority', 'Other Name', 'Additional Information', 'Source(s)', 'Launch Time (UTC)', 'Date Entered/Updated', 'Apogee'])
# Changing formats (str to int or float)
df['Distance Travelled'] = df['Distance Travelled'].apply(lambda x: x.replace('km', '') if isinstance(x, str) else x)
# And then, convert 'Unkown' values to numerics.
df['Distance Travelled'] = df['Distance Travelled'].replace('Unknown', 'NaN') 
# Finally, I can change it to numeric pandas table.
df['Distance Travelled'] = pd.to_numeric(df['Distance Travelled'], errors='coerce').fillna(0)
df['Landing Location'] = df['Landing Location'].apply(lambda x: x.replace('330km east of Hachinohe and 4000 km out into Pacific Ocean', '330km east of Hachinohe') if isinstance(x, str) else x)
# Changing 'location' data
df['Facility Latitude'] = pd.to_numeric(df['Facility Latitude'], errors='coerce')
df['Facility Longitude'] = pd.to_numeric(df['Facility Longitude'], errors='coerce')
# Date changes
df['Date'] = pd.to_datetime(df['Date'])
# Creating a new column for separating the year values in numeric format.
df['Year'] = df['Date'].dt.year
# Another column for changing number values to string values.
df['Month_Number'] = df['Date'].dt.month
# Another column for separating values to month specifically.
df['Month'] = df['Month_Number'].apply(lambda x: calendar.month_abbr[x])
# Another column for counting test numbers.
df["# of Test"] = 1
# Simplifying the Year/Month format.
df['YYYYMM'] = pd.to_datetime(df['Date'], format='%Y%m', errors='coerce')
df['YYYYMM'] = pd.to_datetime(df['Date']).dt.strftime('%Y%m')

Mapping the missile testing facility locations in DPRK

In [2]:
# Utilizing plotly.express, map out the locations with lat/lon data by test frequencies.
test_frequencies = df['Facility Name'].value_counts()
# Once counted values per item, give the match items the same outcomes for mapbox.
df['Frequencies'] = df['Facility Name'].map(test_frequencies)
# Reason for the sorted_df is to configure the legend items by the size.
sorted_df = df.sort_values(by='Frequencies', ascending=False)
# Create the scatter map plot
map_fig2 = px.scatter_mapbox(sorted_df, lat='Facility Latitude', lon='Facility Longitude',
                             color='Facility Name', size='Frequencies',
                             zoom=6, center={'lat': 39.17180415583127, 'lon': 126.551952046587},
                             width=1200, height=700, title='DPRK Missile Facility Locations')
# Configure the text and layout
map_fig2.update_layout(mapbox_style="carto-positron", title_font_size=25, title_font_family="Courier New")
map_fig2.update_layout(margin_autoexpand=True)
# Modify the legend position
map_fig2.update_layout(title=dict(x=0.5, xanchor='center'))
map_fig2.update_layout(margin=dict(t=45, l=0, r=0, b=5))
# Display the scatter map plot
map_fig2.show()